## Import

Import the required library

In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import cv2


import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time

import warnings
warnings.filterwarnings('ignore')

## Read Image files
reading all the images from the dirctory and defining the directory to save the results. A directory called result is created to save the generated results

In [42]:
path = os.path.abspath('../project/project_images')
path_result = os.path.abspath('../project/results')

img_box = path + '/Boxes.png'
img_1 = path + '/Rainier1.png'
img_2 = path + '/Rainier2.png'
img_3 = path + '/Rainier3.png'
img_4 = path + '/Rainier4.png'
img_5 = path + '/Rainier5.png'
img_6 = path + '/Rainier6.png'

img_own_1 = path + '/1.jpg'
img_own_2 = path + '/2.jpg'
img_own_3 = path + '/3.jpg'

img_extra_11 = path + '/Hanging1.png'
img_extra_12 = path + '/Hanging2.png'

img_extra_31 = path + '/ND1.png'
img_extra_32 = path + '/ND2.png'

imgBox = cv2.imread(img_box)
img1 = cv2.imread(img_1)
img2 = cv2.imread(img_2)
img3 = cv2.imread(img_3)
img4 = cv2.imread(img_4)
img5 = cv2.imread(img_5)
img6 = cv2.imread(img_6)

own1 = cv2.imread(img_own_1)
own2 = cv2.imread(img_own_2)
own3 = cv2.imread(img_own_3)

imgex11 = cv2.imread(img_extra_11)
imgex12 = cv2.imread(img_extra_12)

imgex31 = cv2.imread(img_extra_31)
imgex32 = cv2.imread(img_extra_32)

## Get the HarrisCorner
imeplement Harris corner detection algorithm to get the interst points and save the three images into the result folder. Non maximum suppression is also applied. 

In [3]:
def adaptiveNonMaxSup(image, keypoints, num):
    keys_with_radius =[]
    final_result =[]
    for i in range(len(keypoints)):
        key1 = keypoints[i].pt
        lowest_radius = 9999999999
        for j in range(len(keypoints)):
            if(i==j):
                continue
            key2 = keypoints[j].pt
            radius = math.sqrt((key1[0]-key2[0])**2 + (key1[1]-key2[1])**2)
            if(radius < lowest_radius):
                lowest_radius = radius
        keys_with_radius.append([key1[0], key1[1], lowest_radius])
        
    keys_with_radius = sorted(keys_with_radius, key = lambda x :x[2], reverse = True)[:num]
    for key in keys_with_radius:
        final_result.append(cv2.KeyPoint(key[0], key[1], key[2]))
#     outImage = cv2.drawKeypoints(image, final_result,image)
    return final_result

In [4]:
def nonMaxSup(harris):
    harris = np.pad(harris, 1, mode='constant')
    rows, cols = harris.shape
    
    for i in range(1, rows-1, 1):
        for j in range(1, cols-1, 1):
            patch = harris[i-1: i+3, j-1:j+3]
            if(harris[i, j] != patch.max()):
                harris[i,j ]=0
    harris = harris[1: rows, 1:cols]
    return harris

In [5]:
def Corner(img, THRESHOLD):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)
    image *= 1./255
    
    Ix = cv2.Sobel(image, -1,1,0,ksize=3)
    Iy = cv2.Sobel(image, -1,0,1,ksize=3)
    
    Ix2 = Ix**2
    Iy2 = Iy**2
    Ixy = Ix *Iy
    
    Ix2 = cv2.GaussianBlur(Ix2, (3,3), 0)
    Iy2 = cv2.GaussianBlur(Iy2, (3,3), 0)
    Ixy = cv2.GaussianBlur(Ixy, (3,3), 0)
    
    determinant = Ix2*Iy2 - Ixy*Ixy
    trace= Ix2+Iy2
    response = np.divide(determinant,trace)
    response[np.isnan(response)]=0
    magnitude = cv2.sqrt(Ix**2+Iy**2)
    orientations=np.arctan2(Iy, Ix)
    
#     response = nonMaxSup(respons)
    keypoints = np.argwhere(response> THRESHOLD)
    keypoints = [cv2.KeyPoint(x[1],x[0], response[x[0],x[1]])for x in keypoints]
    outImage = cv2.drawKeypoints(img, keypoints,img)
    
#     keyp = adaptiveNonMaxSup(image, response, 4)
    
    return keypoints, magnitude, np.rad2deg(orientations), outImage

## Harris corner Detection (Answer of Project 1a, 1b, and 1c)
Call the harris function to show the image corners and save the result

In [6]:
kp, mg, th, keypoint_box = Corner(imgBox, 2)
keypoints1, image1_magnitude, image1_orientations, keypoint_image1 = Corner(img1,.5)
keypoints2, image2_magnitude, image2_orientations, keypoint_image2 = Corner(img2,.5)

cv2.imshow("keypoint_box", keypoint_box)
cv2.imshow("keypoint_image1", keypoint_image1)
cv2.imshow("keypoint_image2", keypoint_image2)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(path_result+'/1a'+'.png',keypoint_box.astype(np.uint8))
cv2.imwrite(path_result+'/1b'+'.png',keypoint_image1.astype(np.uint8))
cv2.imwrite(path_result+'/1c'+'.png',keypoint_image2.astype(np.uint8))

True

## Extra Credit Keypoint detect Using new detector

In [7]:
import SiftKeypointDetector as newDetect

o, image11_magnitude, image11_orientations, keypoint_image11 = Corner(imgex11, .4) #.5, 1.1
p, image12_magnitude, image12_orientations, keypoint_image12 = Corner(imgex12,.4)

keypoints11 = newDetect.new_impl(imgex11)
keypoints12 = newDetect.new_impl(imgex12)

In [8]:
import SiftKeypointDetector as newDetect

o, image31_magnitude, image31_orientations, keypoint_image31 = Corner(imgex31, .5) #.5, 1.1
p, image32_magnitude, image32_orientations, keypoint_image32 = Corner(imgex32,1.1)

keypoints31 = newDetect.new_impl(imgex31)
keypoints32 = newDetect.new_impl(imgex32)

## Feature Descriptor
get the dominant Angle in the the image patch, 360 degree angles devided into 10 degree  

In [9]:
def getDominantAngle(patch, patch_magnitude):
    angles = np.zeros(36)
    for row in range(len(patch)):
        for col in range(len(patch[0])):
            cell_angle = patch[row, col]
            cell_magnitude = patch_magnitude[row, col]       
            index = int(cell_angle/10)
            angles[index-1] += cell_magnitude
    return angles.argmax()

## Rotation and Contrast invariance for Extra Credits, 

For number 3 extra credit, rotation imvarinace bins ae created 

In [10]:
def rotationInvariance(patch, patch_magnitude):
    angles = np.zeros(18)
    for row in range(len(patch)):
        for col in range(len(patch[0])):
            cell_angle = patch[row, col]
            cell_magnitude = patch_magnitude[row, col]       
            index = int(cell_angle/20)
            angles[index-1] += cell_magnitude
    return angles.argmax()

## Create Feature Vector
crteae 1,128 vector to describe the features of wach keypoint. For now we are ignoring the edges of the imaages. put the orientations as per their magnitudes in corresponding bins

In [11]:
def getDescriptor(interest_point, image_magnitude, image_orientations):
    x, y= interest_point.pt
    top = int(y-8)
    left = int(x-8)
    
    patch = image_orientations[top:top+16, left:left+16]
#     print(patch.shape)
    patch_magnitude = image_magnitude[top:top+16, left:left+16]
    dominant_angle = getDominantAngle(patch, patch_magnitude)
    patch = patch - dominant_angle
    
    feature_vectors = np.zeros(128)
    index =0
    
    for i in range (0,16,4):
        for j in range(0,16,4):
            subpatch = patch[i:i+4, j:j+4]
            bin_1 = ((subpatch>= -180)& (subpatch<-135)).sum()
            bin_2 = ((subpatch>= -135)& (subpatch<-90)).sum()
            bin_3 = ((subpatch>= -90)& (subpatch<-45)).sum()
            bin_4 = ((subpatch>= -45)& (subpatch<0)).sum()
            bin_5 = ((subpatch>= 0)& (subpatch<45)).sum()
            bin_6 = ((subpatch>= 45)& (subpatch<90)).sum()
            bin_7 = ((subpatch>= 90)& (subpatch< 135)).sum()
            bin_8 = ((subpatch>= 135)& (subpatch<180)).sum()
            feature_vectors[index:index+8] = bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8
            
            index +=8
    return feature_vectors

## Get the Descriptor

In [12]:
def getDescriptorForImage(keypoints, magnitudes, oreintations):   
    image_descriptor=[]
    for i in range(len(keypoints)):
        descriptor = getDescriptor(keypoints[i], magnitudes, oreintations)
        image_descriptor.append(descriptor)
    return image_descriptor

In [13]:
def getDistance(feature_vector1, feature_vector2):
    return np.sum((feature_vector1-feature_vector2)**2)

## SSD test 
Get the sum of squared difference of teh descriptor between to images so that we can get a close estimation of correct 

In [14]:
def getSSD(image_descriptor1, image_descriptor2):
    ssd_values =[]
    for i in range(len(image_descriptor1)):
        desc1 = image_descriptor1[i]
        lowest = 999999999999
        lowest_index = -1
        for j in range(len(image_descriptor2)):
            desc2 = image_descriptor2[j]
            distance = getDistance(desc1, desc2)
            if(distance<lowest):
                lowest = distance
                lowest_index =j
        ssd_values.append([lowest, lowest_index])
    return ssd_values

def getSSD2(image_descriptor1, image_descriptor2, ssd_values):
    ssd_values2 =[]
    for i in range(len(image_descriptor1)):
        desc1 = image_descriptor1[i]
        lowest = 999999999999
        lowest_index = -1
        for j in range(len(image_descriptor2)):
            desc2 = image_descriptor2[j]
            distance = getDistance(desc1, desc2)
            if(distance<lowest and distance != ssd_values[i][0]):
                lowest = distance
                lowest_index =j
        ssd_values2.append([lowest, lowest_index])
    return ssd_values2
    

## Ratio Test
put a threshold value to pcik up the good featured of keypoints between two images.

In [15]:
def ratioTest(lowest_ssd, lowest_ssd2, ratio):
    ratio_values=[]
    for i in range(len(lowest_ssd)):
        ssd1 = lowest_ssd[i][0]
        ssd2 = lowest_ssd2[i][0]       
        if(ssd1/ssd2 <= ratio):
            ratio_values.append([ratio, i, lowest_ssd[i][1]])
    return ratio_values

## Contrast Handler 
Use SIFT implementation of Contrast handling

In [16]:
def contrastHandler(desc):
    normalized_list =[]
    for d in desc:
        d = d/(d.sum()+.000000001)
        d = d.clip(0,0.2)
        d = d/d.sum()
        normalized_list.append(d)
    return normalized_list

## Descriptor procedure 
all the descritors are called sequentially and processed in this block since the process requires so many processes 

In [17]:
def runDescriptor(img1, kp1, mag1, orien1,img2, kp2, mag2, orien2, RATIO):
    desc1 = getDescriptorForImage(kp1, mag1, orien1)
    desc2 = getDescriptorForImage(kp2, mag2, orien2)
    image1_desc = contrastHandler(desc1)
    image2_desc = contrastHandler(desc2)
    ssd_values = getSSD(image1_desc, image2_desc)
    ssd_second_values = getSSD2(image1_desc, image2_desc, ssd_values)
    ratio_values = ratioTest(ssd_values, ssd_second_values, RATIO)
    matching_points_ratio =[]
    for i in range(len(ratio_values)):
        matching_points_ratio.append(cv2.DMatch(ratio_values[i][1], ratio_values[i][2], ratio_values[i][0]))

    image3 = cv2.drawMatches(img1, kp1, img2, kp2, matching_points_ratio, None, flags=2)
    cv2.imshow("Matching_points", image3)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return kp1, kp2 , matching_points_ratio, image3


## Run the Descriptor to get the Matches(Answer 2)
Save the result of the matching 

In [18]:
# Rainer Image //.5 and .7, .7,400
key1, key2, good, matched_keypoints =runDescriptor(img1, keypoints1, image1_magnitude, image1_orientations,img2, keypoints2, image2_magnitude, image2_orientations, .7)
cv2.imwrite(path_result+'/2'+'.png',matched_keypoints.astype(np.uint8))

True

## READ THE IMAGES AGAIN
--this is done mainly for debugging purpose otherwise it might produce some distoted image

In [19]:
path = os.path.abspath('../project/project_images')
path_result = os.path.abspath('../project/results')

img_1 = path + '/Rainier1.png'
img_2 = path + '/Rainier2.png'
img_3 = path + '/Rainier3.png'
img_4 = path + '/Rainier4.png'
img_5 = path + '/Rainier5.png'
img_6 = path + '/Rainier6.png'

img_own_1 = path + '/1.jpg'
img_own_2 = path + '/2.jpg'
img_own_3 = path + '/3.jpg'

img_extra_11 = path + '/Hanging1.png'
img_extra_12 = path + '/Hanging2.png'

img_extra_31 = path + '/ND1.png'
img_extra_32 = path + '/ND2.png'

img1 = cv2.imread(img_1)
img2 = cv2.imread(img_2)
img3 = cv2.imread(img_3)
img4 = cv2.imread(img_4)
img5 = cv2.imread(img_5)
img6 = cv2.imread(img_6)

own1 = cv2.imread(img_own_1)
own2 = cv2.imread(img_own_2)
own3 = cv2.imread(img_own_3)

imgex11 = cv2.imread(img_extra_11)
imgex12 = cv2.imread(img_extra_12)

imgex31 = cv2.imread(img_extra_31)
imgex32 = cv2.imread(img_extra_32)

# Project Implementation RANSAC and Stitching (Answer 3 and 4)

First Compute the SIFTfeature descriptor using opencv Library. for a pair of images this takes as a parameter the two images and Returns the matches found and the keypoints for both the images (since for project it is not mandatory to use own implemntation, since my own implementation gives a correct output but takes a lot of time, so i preferred using opencv)

#### function match: params(image 1 and image 2), returns number of matches and keypoint of both image

In [20]:
def match(img1,img2):
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    matcher = cv2.BFMatcher()
    raw_matches = matcher.knnMatch(des1, des2, k=2)
    good_matches=[]
    good_points=[]
    for m1, m2 in raw_matches:
        if m1.distance < .4 * m2.distance:
            good_points.append((m1.trainIdx, m1.queryIdx))
            good_matches.append(m1)
    return good_matches, kp1, kp2

Project points from image one to another image given a Homography, returs the projected image 

In [21]:
def project(x1, y1, H):
    w = (H[2][0]*x1 + H[2][1]*y1 + H[2][2])
    if (w == 0):
        w == 0.00000001
    x2 = np.float32(((H[0][0]*x1 + H[0][1]*y1 + H[0][2])/w))
    y2 = np.float32(((H[1][0]*x1 + H[1][1]*y1 + H[1][2])/w))
    return x2, y2

## Compute Inliers , threshold given
computeInlierCount is a helper function for RANSAC that computes the number of inlying points given a homography "H". That is, project the first point in each match using the function "project". If the projected point is less than the distance "inlierThreshold" from the second point, it is an inlier

In [22]:
def computeInlierCount(H, matches, inlierThreshold, keypoint1, keypoint2): 
    numMatches = 0
    inliers=[]
    for m in range(len(matches)):
        x2, y2 = project(keypoint1[matches[m].queryIdx].pt[0], keypoint1[matches[m].queryIdx].pt[1], H)
        x1 = keypoint2[matches[m].trainIdx].pt[0]
        y1 = keypoint2[matches[m].trainIdx].pt[1]
        distance = math.sqrt(((x1-x2)**2) + ((y1-y2)**2))
        if(distance < inlierThreshold):
            numMatches+= 1
            inliers.append(matches[m])
    return numMatches, inliers

## RANSAC Impelemntation 
<p> This function takes a list of potentially matching points between two images and returns the homography transformation that relates them </br>
    <p> a. For "numIterations" iterations: 
        i. Randomly select 4 pairs of potentially matching points from "matches".</p>
        ii. Compute the homography relating the four selected matches </p>
        iii. If this homography produces the highest number of inliers, store it as the
             best homography.</p></br>
    <p> b. Given the highest scoring homography, once again find all the inliers. Compute a
        new refined homography using all of the inliers (not just using four points as you
        did previously. ) Compute an inverse homography as well, and return their values in
        "hom" and "homInv".

In [23]:
def RANSAC (matches, numIterations, inlierThreshold,  keypoint1, keypoint2):
    maxNumInliers = 0
    finalSrc =[]
    finalDst =[]
    for i in range(numIterations):
        srcPoints =[]
        dstPoints =[]
        
        four_random_matches = random.sample(matches, 4)
#         print(four_random_matches//dmatch)
        
        for j in range(len(four_random_matches)):
#             four_random_matches[i].trainIdx
            srcPoints.append(keypoint1[four_random_matches[j].queryIdx].pt)
            dstPoints.append(keypoint2[four_random_matches[j].trainIdx].pt)
        src = np.float32(srcPoints).reshape(-1,1,2)
        dst = np.float32(dstPoints).reshape(-1,1,2)
        
        H, status = cv2.findHomography(src, dst, 0)
        if(H is None):
            continue
        
        numMatches, inliers = computeInlierCount(H, matches, inlierThreshold, keypoint1, keypoint2)
        if (numMatches > maxNumInliers):
            maxNumInliers = numMatches
            hom = H
        
    numMatchesFinal, inliersFinal = computeInlierCount(hom, matches, inlierThreshold,keypoint1, keypoint2)
    
    for k in range(len(inliersFinal)):
        finalSrc.append(keypoint1[inliersFinal[k].queryIdx].pt)
        finalDst.append(keypoint2[inliersFinal[k].trainIdx].pt)
        
    srcF = np.float32(finalSrc).reshape(-1,1,2)
    dstF = np.float32(finalDst).reshape(-1,1,2)
        
    hom, status = cv2.findHomography(srcF, dstF,0)
    homInv = np.float32(np.linalg.inv(hom))
    
    return hom, homInv,inliersFinal

## Stitch function returns the panorama and blendimage1 and blendImage2 (to be used in blending)
Takes in the two images and the computed homography and inverse hoography . then computes the size of the stitched image given the two images then for each point of the image project
Stitch the images together using the computed homography 

In [24]:
def stitch(img1, img2, hom, homInv):
    h1, w1 = img1.shape[:2]
    height, width = img2.shape[:2]
    corners =[]
    
    corners.append(project(0,0, homInv))
    corners.append(project(0, height - 1, homInv))
    corners.append(project(width - 1, height - 1, homInv))
    corners.append(project(width - 1, 0, homInv))
    
#     result = cv2.warpPerspective(img2, homInv,(img2.shape[1] + img1.shape[1], img2.shape[0]))
#     cv2.imshow("Scanned Image", result)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    corners = np.float32( np.array(corners))
    
    (minX, maxX, minY, maxY )= (0, w1, 0, h1)
    
    for l in range(len(corners)):
        if ((corners[l][0]) < minX):
            minX = math.floor(corners[l][0])
        elif((corners[l][0]) > maxX):
            maxX = math.ceil(corners[l][0])
            
        if((corners[l][1]) < minY):
            minY = math.floor(corners[l][1])
        elif((corners[l][1]) > maxY):
            maxY = math.ceil(corners[l][1])

    minX = abs(minX)
    minY = abs(minY)
    
    stitchedRow = (minY + maxY)
    stitchedCol = (maxX + minX)

    stitched = np.zeros([stitchedRow,stitchedCol, 3])
    blendImg1 = np.zeros(stitched.shape)
    blendImg2 = np.zeros(stitched.shape)
    temp = np.zeros(stitched.shape)

    
    for h in range(h1):
        for w in range(w1):
            if img1[h][w][0] != 0 or img1[h][w][1] != 0 or img1[h][w][2] != 0:
                stitched[h+minY][w + minX]  = img1[h][w]
                temp[h+minY][w + minX]  = img1[h][w]
                
    for a in range(stitchedRow):
        for b in range(stitchedCol):
            xs , ys = project(b-minX, a-minY, hom)
            xs = xs.astype(int)
            ys = ys.astype(int)
            if((temp[a][b]).all() !=0):
                blendImg1[a][b] = temp[a][b]
            
            if(xs > 0 and xs < width and ys > 0 and ys < height):
                stitched[a][b] = img2[ys][xs] 
                stitched[a][b] = cv2.getRectSubPix(img2, (1,1), (xs,ys)) 
                if((stitched[a][b]).all() !=0):
                    blendImg2[a][b] = stitched[a][b] 
    
    return stitched, blendImg1, blendImg2

## Run the Ransac using two rainer image and save the final inlier (Answer 3)'
at this point there is not a single wrong match. if there exists a wrong match the panorama will not be formed properly. 
NOTE: the number of inliers seems to be a bit higher because, the matching is calculated using slef implemntation while the inliers are calculated using cv.sift

In [25]:
good_matches,kp1,kp2 = match(img1,img2)
hom, homInv,inliersFinal  = RANSAC(good_matches,400,.6,kp1,kp2)
image3 = cv2.drawMatches(img1,kp1, img2, kp2, inliersFinal, None, flags=2)
cv2.imwrite(path_result+'/3'+'.png',image3.astype(np.uint8))
cv2.imshow("Inliers", image3)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Create an Stitched Image using two rainer image (Answer 4)
Finally create a stiched image, 
#### also run all stitched and save the image with six rainer image (4a), 
take all the images in an array and call all the function inside a for loop, each step is saved for the progress

In [26]:
i =0
pano = []
panorama, blend1,blend2= stitch(img1, img2, hom, homInv)
pano.append(panorama.astype(np.uint8))
cv2.imwrite(path_result+'/4.png',panorama.astype(np.uint8))
cv2.imshow("Stitched", pano[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

imgs = [img1, img2, img3, img4, img5, img6]
for m in range(len(imgs)):
    if(m<(len(imgs)-2)):
        good_matches,kp1,kp2 = match(pano[m],imgs[m+2])
        hom, homInv,inliersFinal  = RANSAC(good_matches,400,.7,kp1,kp2)
        panorama, b1,b2 = stitch(pano[m],imgs[m+2], hom, homInv)
        pano.append(panorama.astype(np.uint8))
        cv2.imwrite(path_result+'/steps_stitched%d.jpg' % (i),panorama.astype(np.uint8))
        i+=1
cv2.imwrite(path_result+'/Allstitched.png' ,pano[4].astype(np.uint8))
cv2.imshow("AllStitched", pano[4].astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

## use all the algorithm to stitch two images taken by myself
//Takes a lot of time to run, sift.detectandcompute did not do a good job in creating the stitch. so used corner detection of harris and then used sift.compute to get the desciptor.

In [27]:
def harris_own(img):
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    dst = cv2.cornerHarris(gray_img.astype(np.float32), 2, 3, 0.04)
    result_img = img.copy() # deep copy image

    # Threshold for an optimal value, it may vary depending on the image.
    result_img[dst > 0.01 * dst.max()] = [0, 255, 0]

    # for each dst larger than threshold, make a keypoint out of it
    keypoints = np.argwhere(dst > 0.01 * dst.max())
    keypoints = [cv2.KeyPoint(x[1], x[0], 1) for x in keypoints]

    return (keypoints, result_img)

def match_own(img1,img2, kp1, kp2):
    sift = cv2.xfeatures2d.SIFT_create()
    a, des1 = sift.compute(img1,kp1)
    b, des2 = sift.compute(img2,kp2)
    matcher = cv2.BFMatcher()
    raw_matches = matcher.knnMatch(des1, des2, k=2)
    good_points = []
    good_matches=[]
    for m1, m2 in raw_matches:
        if m1.distance < .34 * m2.distance:
            good_points.append(cv2.DMatch(m1.trainIdx, m1.queryIdx, 1))
            good_matches.append(m1)
    return good_matches

In [28]:
kpo1 ,ds1= harris_own(own1)
kpo2 ,ds2= harris_own(own2)
kpo3 ,ds3= harris_own(own3)

imgo1 = cv2.imread(img_own_1)
imgo2 = cv2.imread(img_own_2)
imgo3 = cv2.imread(img_own_3)

good_matches_o = match_own(imgo1,imgo2, kpo1, kpo2)
hom_o, homInv_o,inliersFinal_o  = RANSAC(good_matches_o,100,.2,kpo1,kpo2)
pano_o, b1,b2= stitch(own1, own2, hom_o, homInv_o)
cv2.imwrite(path_result+'/ownstep2.jpg',pano_o.astype(np.uint8))

a = cv2.imread(path_result + '/ownstep2.jpg')
kpos ,ds4 = harris_own(a)
own = cv2.imread(path_result + '/ownstep2.jpg')
good_matches_o2 = match_own(own ,imgo3, kpos, kpo3)
hom_o2, homInv_o2,inliersFinal_o2  = RANSAC(good_matches_o2,300,.2,kpos,kpo3)
pano_o2, b1,b2= stitch(own, own3, hom_o2, homInv_o2)
cv2.imwrite(path_result+'/ownStitched.png',pano_o2.astype(np.uint8))

cv2.imshow("ownStitched", pano_o2.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

## Extra Credit (#4)
### Image Blending 
take the warped image for the stich function output and belnd the images using laplacian pyramid building. I followed the procedure of thoery, taking the gaussian, then down sampling then subtracting the previous layer output for the next. then creating a laplacian filter and then blending them. For beautifying the top 64px of the stitched image is cropped since that portion was black

In [29]:
def blending(img1, img2):
    levels = 6
    gaussianImg1 = [img1.astype(np.float32)]
    gaussianImg2 = [img2.astype(np.float32)]
    for i in range(levels):
        img1 = cv2.pyrDown(img1).astype(np.float32)
        gaussianImg1.append(img1)
        img2 = cv2.pyrDown(img2).astype(np.float32)
        gaussianImg2.append(img2)
    laplacianImg1 = [gaussianImg1[levels]]
    laplacianImg2 = [gaussianImg2[levels]]

    for i in range(levels,0,-1):
        temp = cv2.pyrUp(gaussianImg1[i]).astype(np.float32)
        temp = cv2.resize(temp, (gaussianImg1[i-1].shape[1],gaussianImg1[i-1].shape[0]))
        laplacianImg1.append(gaussianImg1[i-1]-temp)

        temp = cv2.pyrUp(gaussianImg2[i]).astype(np.float32)
        temp = cv2.resize(temp, (gaussianImg2[i-1].shape[1],gaussianImg2[i-1].shape[0]))
        laplacianImg2.append(gaussianImg2[i-1]-temp)

    laplacianList = []
    for lpImg1,lpImg2 in zip(laplacianImg1,laplacianImg2):
        rows,cols = lpImg1.shape[:2]
        mask1 = np.zeros(lpImg1.shape,dtype = np.float32)
        mask2 = np.zeros(lpImg2.shape,dtype = np.float32)
        mask1[:, 0:int(cols/ 2)] = 1
        mask2[:,int( cols / 2):] = 1

        temp1 = lpImg1 * mask1
        temp2 = lpImg2 * mask2
        temp = temp1 + temp2
        
        laplacianList.append(temp)
    
    blend = laplacianList[0]
    for i in range(1,levels+1):
        blend = cv2.pyrUp(blend)   
        blend = cv2.resize(blend, (laplacianList[i].shape[1],laplacianList[i].shape[0]))
        blend = blend+ laplacianList[i]
    
    np.clip(blend, 0, 255, out=blend)
    crop = blend[64:(64+len(blend)), 0:(0+len(blend[0]))]
    cv2.imwrite(path_result+'/BlendedImage.png',crop.astype(np.uint8))
    cv2.imshow("Blended", crop.astype(np.uint8))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

blending (blend1,blend2)


## EXTRA Credit ()

A new keypoint detector implementing SIFT to detect the keypoints, used MOPS rotation invariance algorithm to describe

## Rotated image ransac, keypoint and sticthing
the best result is found with 1000 iterations and inlierthreshold .09 and the ratio is .9)

In [30]:
key11, key12, good1, matched_keypoints1 = runDescriptor(imgex11, keypoints11, image11_magnitude, image11_orientations,
              imgex12, keypoints12, image12_magnitude, image12_orientations, .9)
good1, key11, key12 = match(imgex11,imgex12)
cv2.imwrite(path_result+'/rotated_matching'+'.png',matched_keypoints1.astype(np.uint8))

True

In [31]:
hom1, homInv1,inliersFinal1  = RANSAC(good1,100,.09,key11,key12)
image12 = cv2.drawMatches(imgex11,key11, imgex12, key12, inliersFinal1, None, flags=2)
cv2.imwrite(path_result+'/rotated_matching_point_inliers'+'.png',image12.astype(np.uint8))
cv2.imshow("rotated_matching_point_inliers", image12)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
panorama1, _,_= stitch(imgex11, imgex12, hom1, homInv1)
cv2.imwrite(path_result+'/rotated_stitched.png',panorama1.astype(np.uint8))
cv2.imshow("rotated_stitched", panorama1.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

## OLD image ransac, keypoint and sticthing
the best result is found with 600 iterations and inlierthreshold .6 and the ratio is .75

In [33]:
key31, key32, good3, matched_keypoints3 = runDescriptor(imgex31, keypoints31, image31_magnitude, image31_orientations,
              imgex32, keypoints32, image32_magnitude, image32_orientations, .75)
cv2.imwrite(path_result+'/old_matching'+'.png',matched_keypoints3.astype(np.uint8))

True

In [40]:
hom3, homInv3,inliersFinal3  = RANSAC(good3,600,.7,key31,key32)
image13 = cv2.drawMatches(imgex31,key31, imgex32, key32, inliersFinal3, None, flags=2)
cv2.imwrite(path_result+'/old_matching_point_inliers'+'.png',image13.astype(np.uint8))
cv2.imshow("old_matching_point_inliers", image13)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
## with the same values of thrshold and iterations the Stitched image sometimes becomes bit different form each other,
##issue can be solved by running the previous block again
panorama3, _,_= stitch(imgex31, imgex32, hom3, homInv3)
cv2.imwrite(path_result+'/old_stitched.png',panorama3.astype(np.uint8))
cv2.imshow("old_stitched", panorama3.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
# Reference:
# http://www.cs.utoronto.ca/~urtasun/courses/CV/lecture04.pdf
# https://people.csail.mit.edu/hasinoff/320/sift-notes.txt
# https://courses.cs.washington.edu/courses/cse455/09wi/Lects/lect6.pdf
# https://github.com/geekyspartan/Image-stitching-panorama
# https://github.com/agjayant/PanoramaStitching/blob/master/
# https://www.csie.ntu.edu.tw/~cyy/courses/vfx/12spring/lectures/handouts/lec04_stitching_4up.pdf
# https://docs.opencv.org/master/d1/de0/tutorial_py_feature_homography.html
# https://courses.cs.washington.edu/courses/cse576/13sp/projects/project1/artifacts/tanvir/index.html
# https://medium.com/@lerner98/implementing-sift-36c619df7945
# https://kushalvyas.github.io/stitching.html
# https://dsp.stackexchange.com/questions/4893/scale-and-rotation-invariant-feature-descriptors